In [ ]:
!pip install gradio transformers

In [ ]:
import threading
import gradio as gr
from transformers import pipeline

In [ ]:
!pip install faster_whisper

In [ ]:
from transformers import AutoTokenizer

In [ ]:
!pip install deep_translator

In [ ]:
import time

In [ ]:
!pip install sentencepiece==0.1.99
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-jap")
from transformers import MarianTokenizer, MarianMTModel
#import whisper
import time
from faster_whisper import WhisperModel
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM


from deep_translator import GoogleTranslator

en_to_ja_model_name = "Helsinki-NLP/opus-mt-en-jap"
en_to_ja_tokenizer = MarianTokenizer.from_pretrained(en_to_ja_model_name)
en_to_ja_model = MarianMTModel.from_pretrained(en_to_ja_model_name)

# pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-mul")
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mul")
# model_trans = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-mul")
time_store=0
def translate_en_to_ja(audio,choice,state1_en_to_ja="",state2_ja_to_en=""):
    global time_store
    func_start_time = time.time()
    print(choice)
    if choice == "Japanese":
        model = "medium.en"
        text1 = ""
        audio_model = WhisperModel(model, device="cpu", compute_type="int8")
        text, _ = audio_model.transcribe(audio)
        for segment in text:
            text1 = segment.text
        text = text1
        file_append= open('sample.txt','a')
        file_append.write(text)
        file_append.close()
        print("English:", text)
        #translated_text = GoogleTranslator(source='auto', target='ja').translate(text)
        inputs = en_to_ja_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        translation = en_to_ja_model.generate(**inputs, max_length=512)
        translated_text = en_to_ja_tokenizer.decode(translation[0], skip_special_tokens=True)

        # inputs = tokenizer.encode(text, return_tensors="pt", truncation=True)
        # translation = model_trans.generate(inputs, max_length=128, num_beams=4, early_stopping=True, target_lang="ja")
        # translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
        state1_en_to_ja += translated_text + " "
        print("Japanese:", state1_en_to_ja)
        time_store+=time.time()-func_start_time
        file_append= open('time.txt','a')
        file_append.write("  "+str(time_store))
        file_append.close()

        return state1_en_to_ja,"", state1_en_to_ja,""
    else:
        model = "small"
        text1 = ""
        audio_model = WhisperModel(model, device="cpu", compute_type="int8")
        text, _ = audio_model.transcribe(audio, task="translate")
        for segment in text:
            text1 = segment.text
        text = text1
        print("Japanese transcript:", text)
        state2_ja_to_en+= text + " "
        print("English translate:", state2_ja_to_en)
        return "",state2_ja_to_en,"",state2_ja_to_en

# Create Gradio interface
demo = gr.Blocks(live=True)
with gr.Blocks() as demo:
    state1_en_to_ja = gr.State(value="")
    state2_ja_to_en = gr.State(value="")
    with gr.Row():
        with gr.Column():
            audio_en_to_ja = gr.Audio(source="microphone", type="filepath", streaming=True)
            radio_button = gr.Radio(["Japanese", "English"], label="Choose in which language you want to translate", value="Japanese")
            liv_translation_en_to_ja = gr.Textbox(label="Translation (EN to JA)")
            liv_translation_ja_to_en = gr.Textbox(label="Translation (JA to EN)")
            audio_en_to_ja.stream(
                fn=translate_en_to_ja,
                inputs=[audio_en_to_ja, radio_button,state1_en_to_ja,state2_ja_to_en],
                outputs=[liv_translation_en_to_ja, liv_translation_ja_to_en,state1_en_to_ja,state2_ja_to_en]
            )

demo.launch()